In [1]:
# Update only this cell with your local parameters

predictions_folder = "ChildrensPredictionArrays"
ground_truth_file  = r"ChildrensTestArrays\segmentation-test.npy"

acceptable_margin_mm = 1.0
mm_per_pixel = 1.0

roc_thresholds = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1,
                  0.08, 0.06, 0.04, 0.02, 0.01,
                  0.008, 0.006, 0.004, 0.002, 0.001,
                  0.0008, 0.0006, 0.0004, 0.0002, 0.0001,
                  0.00001, 0.000001]


In [2]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os 
import scipy.ndimage

from random import sample

from evaluation_metrics import *

from local_vars import root_folder

In [3]:
# Read data arrays

predictions_folder_fullpath = os.path.join(root_folder, predictions_folder)

predictions_file_list = [f for f in os.listdir(predictions_folder_fullpath) if f.endswith('.npy')]
num_predictions = len(predictions_file_list)

print("Found {} predictions in {}".format(num_predictions, predictions_folder_fullpath))

groundtruth_fullname = os.path.join(root_folder, ground_truth_file)

print("Reading ground truth from:  {}".format(groundtruth_fullname))

groundtruth_data = np.load(groundtruth_fullname)
num_groundtruth = groundtruth_data.shape[0]

print("\nFound {} ground truth images".format(num_groundtruth))

Found 8 predictions in d:\Data\ChildrensPredictionArrays
Reading ground truth from:  d:\Data\ChildrensTestArrays\segmentation-test.npy

Found 1126 ground truth images


In [ ]:
for prediction_filename in predictions_file_list:
    prediction_fullname = os.path.join(predictions_folder_fullpath, prediction_filename)
    prediction_data = np.load(prediction_fullname)
    
    true_positives, false_positives = compute_roc(
        roc_thresholds, prediction_data, groundtruth_data, acceptable_margin_mm, mm_per_pixel)
    
    best_threshold_index = compute_goodness(roc_thresholds, true_positives, false_positives)
    print("\nMetrics for file: {}".format(prediction_fullname))

    print("  Best threshold:           {}".format(roc_thresholds[best_threshold_index]))
    print("  Best true positive rate:  {}".format(true_positives[best_threshold_index]))
    print("  Best false positive rate: {}\n".format(false_positives[best_threshold_index]))



Metrics for file: d:\Data\ChildrensPredictionArrays\q000_prediction.npy
  Best threshold:           0.0004
  Best true positive rate:  0.9039675831551732
  Best false positive rate: 0.04544124769377404


Metrics for file: d:\Data\ChildrensPredictionArrays\q001_prediction.npy
  Best threshold:           0.0002
  Best true positive rate:  0.9380002983145229
  Best false positive rate: 0.053733833277659925


Metrics for file: d:\Data\ChildrensPredictionArrays\q002_prediction.npy
  Best threshold:           0.002
  Best true positive rate:  0.9453835827574207
  Best false positive rate: 0.05730193953025818


Metrics for file: d:\Data\ChildrensPredictionArrays\q003_prediction.npy
  Best threshold:           0.0006
  Best true positive rate:  0.9545070352508328
  Best false positive rate: 0.04775410274776092


Metrics for file: d:\Data\ChildrensPredictionArrays\q004_prediction.npy
  Best threshold:           0.0002
  Best true positive rate:  0.9277333068164867
  Best false positive rate: 0